In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
  LoraConfig ,
  PeftConfig ,
  PeftModel ,
  get_peft_model ,
  prepare_model_for_kbit_training,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
notebook_login

<function huggingface_hub._login.notebook_login(new_session: bool = True, write_permission: bool = False) -> None>

### Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ayush-vatsal/description_to_caption")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# MODEL_NAME = "tiiuae/falcon-7b-instruct"
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_use_double_quant = True,
  bnb_4bit_quant_type= "nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
def print_trainable_parameters(model):

  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(f"trainable_params: {trainable_params} || all_params: {all_param} || Percentage of trainable params: {100*trainable_params / all_param}")

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable_params: 4718592 || all_params: 3613463424 || Percentage of trainable params: 0.13058363808693696


### Baseline Perfomance check, Inference before finetuning

In [ ]:
prompt = f"""
Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption:
""".strip()
print(prompt)

Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption:


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 50
generation_config.temperature = 0.01
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id=tokenizer.eos_token_id
generation_config.eos_token_id=tokenizer.eos_token_id

In [ ]:
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors = "pt").to(device)
with torch.inference_mode():
  outputs =model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption: Taking a moment to reflect and appreciate the beauty of solitude.


Convert the given image description to social media worthy caption
Description: A man walking alone in the road

Caption: Taking a moment to reflect and appreciate the beauty of solitude.

### Prepare Dataset

In [ ]:
dataset["train"][0]

{'Description': 'A child in a pink dress is climbing up a set of stairs in an entryway.',
 'Caption': '"Every step is a little victory on the path to greatness. This little adventurer knows no limits. 💗🌟"'}

In [ ]:
def generate_prompt(data):
  return f"""
Convert the given image description to social media worthy caption:
Description: {data["Description"]}
Caption: {data["Caption"]}
""".strip()

def generate_and_tokenize_prompt(data):
  full_prompt = generate_prompt(data)
  tokenized_full_prompt = tokenizer(full_prompt, padding = True, truncation = True)
  return tokenized_full_prompt

In [ ]:
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Description', 'Caption', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 121
})

## Let's start training

In [ ]:
OUTPUT_DIR =  "experiments"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7712), started 0:01:05 ago. (Use '!kill 7712' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
training_args = transformers.TrainingArguments(
        num_train_epochs=30,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=80,
        learning_rate=2e-4,
        save_total_limit = 3,
        fp16=True,
        logging_steps=1,
        output_dir=OUTPUT_DIR,
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
        report_to = "tensorboard",
)

trainer = transformers.Trainer(
    model = model,
    train_dataset = dataset,
    args = training_args,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False),
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.629600
2,3.680400
3,3.619100
4,3.615600
5,3.453300
6,3.358600
7,3.100500
8,2.959800
9,2.820200
10,2.721500


TrainOutput(global_step=80, training_loss=1.5848846666514873, metrics={'train_runtime': 1575.2584, 'train_samples_per_second': 3.25, 'train_steps_per_second': 0.051, 'total_flos': 7720070399867904.0, 'train_loss': 1.5848846666514873, 'epoch': 40.0})

In [ ]:
model.save_pretrained("trained_model")

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub(
    "ayush-vatsal/caption_qlora_finetune", use_auth_token = True
)

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayush-vatsal/caption_qlora_finetune/commit/132d639713240b065bca1957fe4d7ee619c4b986', commit_message='Upload model', commit_description='', oid='132d639713240b065bca1957fe4d7ee619c4b986', pr_url=None, pr_revision=None, pr_num=None)

### Load Finetuned Model

In [ ]:
peft_model = "ayush-vatsal/caption_qlora_finetune"
config = PeftConfig.from_pretrained(peft_model)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict = True,
    quantization_config=bnb_config,
    device_map= "auto",
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, peft_model)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 50
generation_config.temperature = 0.01
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id=tokenizer.eos_token_id
generation_config.eos_token_id=tokenizer.eos_token_id

In [ ]:
prompt = f"""
Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption:
""".strip()
print(prompt)

Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption:


In [ ]:
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors = "pt").to(device)
with torch.inference_mode():
  outputs =model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Convert the given image description to social media worthy caption
Description: A man walking alone in the road
Caption: "In the company of his own thoughts, this man navigates the roads of life. Each step brings him closer to his destination, illuminated by the wisdom he carries within. 🌟🚶"
User 
